<h2> Taller 8 - Pick and Place</h2>

<br>Requiere que los archivos 'sim.py', 'simConst.py', 'remoteapi.dll' estén alojados en la misma carpeta que este cuaderno de Jupyter.
<br> Desde CoppeliaSim, abrir la escena MTB_Pick_N_Place.ttt


In [1]:
# importamos las librerías necesarias
import sim          # librería para conectar con CoppeliaSim
import sympy as sp  # librería para cálculo simbólico
import time

### 0. Verificamos que todo esté funcionando... 

El robot cuenta con una vensosa en el extremo. Ésta se activa automáticamente al acercarse al cubo (1 mm o menos) y se desactiva cuando el cubo se coloca sobre la plataforma (5 mm o menos).

In [2]:
def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID


In [3]:
def setEffector(val):
# acciona el efector final
# val es Int con valor 0 ó 1 para desactivar o activar el actuador final.
    res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
        "suctionPad", sim.sim_scripttype_childscript,"setEffector",[val],[],[],"", sim.simx_opmode_blocking)
    return res

In [4]:
# Requerimos los manejadores para las articulaciones y el Dummy
clientID = connect(19999)

retCode,effector=sim.simxGetObjectHandle(clientID,'effector',sim.simx_opmode_blocking)
retCode,joint1=sim.simxGetObjectHandle(clientID,'MTB_joint1',sim.simx_opmode_blocking)
retCode,joint2=sim.simxGetObjectHandle(clientID,'MTB_joint2',sim.simx_opmode_blocking)
retCode,joint3=sim.simxGetObjectHandle(clientID,'MTB_joint3',sim.simx_opmode_blocking)
retCode,joint4=sim.simxGetObjectHandle(clientID,'MTB_joint4',sim.simx_opmode_blocking)
retCode,caja1=sim.simxGetObjectHandle(clientID,'Caja1',sim.simx_opmode_blocking)
retCode,caja2=sim.simxGetObjectHandle(clientID,'Caja2',sim.simx_opmode_blocking)
retCode,caja3=sim.simxGetObjectHandle(clientID,'Caja3',sim.simx_opmode_blocking)

print(effector, joint1, joint2, joint3, joint4, caja1, caja2, caja3)

('conectado a', 19999)
(38, 21, 22, 23, 34, 35, 36, 37)


In [9]:
# Enviamos las posiciones a las articulaciones
q = [0, 0, 0.0]

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_oneshot)


In [10]:
# Activamos o desactivamos el actuador final
setEffector(0)

0

###  1. Preparamos la información necesaria
A partir de la matriz de cinemática directa calculamos la cinemática inversa.

In [11]:
# 1. Las expresiones de cinemática directa

# En una actividad anterior se obtuvo la matriz de cinemática directa
# para el actuador final, como función de las posiciones de las articulaciones.

q1 = sp.symbols('q1') # ángulo de la articulación rotacional joint1, en radianes
q2 = sp.symbols('q2') # ángulo de la articulación rotacional joint2, en radianes
q3 = sp.symbols('q3') # posición de la articulación prismática joint3, en metros
q4 = sp.symbols('q4') # ángulo de la articulación rotacional joint4, en radianes

T = sp.Matrix([[sp.cos(q1 + q2 + q4), -sp.sin(q1 + q2 + q4), 0, 0.467*sp.cos(q1) + 0.4005*sp.cos(q1 + q2)],
            [sp.sin(q1 + q2 + q4), sp.cos(q1 + q2 + q4), 0, 0.467*sp.sin(q1) + 0.4005*sp.sin(q1 + q2)], 
            [0, 0, 1, 0.234 - q3], 
            [0, 0, 0, 1]])
T

Matrix([
[cos(q1 + q2 + q4), -sin(q1 + q2 + q4), 0, 0.467*cos(q1) + 0.4005*cos(q1 + q2)],
[sin(q1 + q2 + q4),  cos(q1 + q2 + q4), 0, 0.467*sin(q1) + 0.4005*sin(q1 + q2)],
[                0,                  0, 1,                          0.234 - q3],
[                0,                  0, 0,                                   1]])

###  2. Obtenemos la posición y orientación del punto de destino


In [12]:
# Definimos una función para construir la matriz de rotación
# a partir de los ángulos de euler

def matrixFromEuler(alpha, beta, gamma):
    # theta y alpha en radianes
    # d y a en metros
    Ra = sp.Matrix([[1, 0, 0, 0],
                   [0, sp.cos(alpha), -sp.sin(alpha), 0],
                   [0, sp.sin(alpha), sp.cos(alpha), 0],
                   [0, 0, 0, 1]])
    Rb = sp.Matrix([[sp.cos(beta), 0, sp.sin(beta), 0],
                   [0, 1, 0, 0],
                   [-sp.sin(beta), 0, sp.cos(beta), 0],
                   [0, 0, 0, 1]])
    Rc = sp.Matrix([[sp.cos(gamma), -sp.sin(gamma), 0, 0],
                   [sp.sin(gamma), sp.cos(gamma), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    T = Ra*Rb*Rc
    return T

In [16]:
matrixFromEuler(45*sp.pi/180, 0, 45*sp.pi/180)

Matrix([
[sqrt(2)/2, -sqrt(2)/2,          0, 0],
[      1/2,        1/2, -sqrt(2)/2, 0],
[      1/2,        1/2,  sqrt(2)/2, 0],
[        0,          0,          0, 1]])

In [17]:
# A partir de las coordenadas de posición y los ángulos de Euler
# es posible obtener la matriz de transformación

x = 0.5
y = 0.5
z = 0.1
alpha = 0
beta = 0
gamma = 45*sp.pi/180

t = sp.Matrix([[1, 0, 0, x],
               [0, 1, 0, y], 
               [0, 0, 1, z], 
               [0, 0, 0, 1]])

D = t*matrixFromEuler(alpha, beta, gamma)
D

Matrix([
[sqrt(2)/2, -sqrt(2)/2, 0, 0.5],
[sqrt(2)/2,  sqrt(2)/2, 0, 0.5],
[        0,          0, 1, 0.1],
[        0,          0, 0,   1]])

###  3. Calculamos ahora la cinemática inversa


In [18]:
# La función nsolve de sympy busca valores busca soluciones que
# igualan una expresión a cero. Si requerimos T = D, entonces buscamos T-D = 0

T-D


Matrix([
[cos(q1 + q2 + q4) - sqrt(2)/2, -sin(q1 + q2 + q4) + sqrt(2)/2, 0, 0.467*cos(q1) + 0.4005*cos(q1 + q2) - 0.5],
[sin(q1 + q2 + q4) - sqrt(2)/2,  cos(q1 + q2 + q4) - sqrt(2)/2, 0, 0.467*sin(q1) + 0.4005*sin(q1 + q2) - 0.5],
[                            0,                              0, 0,                                0.134 - q3],
[                            0,                              0, 0,                                         0]])

In [19]:
# Buscamos una solución relajando la precisión requerida para facilitar el cálculo.
# La precisión por defecto es de 15 cifras significativas.

try:
    q = sp.nsolve((T-D), (q1, q2, q3, q4), (1, 1, 1, 1), prec=6)
except:
    print('no se encontró la solución')
    q = [0, 0, 0, 0]
q

Matrix([
[ 0.220089],
[  1.23996],
[    0.134],
[-0.674648]])

In [20]:
# enviamos los ángulos a las articulaciones
clientID = connect(19999)
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint4, q[3], sim.simx_opmode_blocking)


('conectado a', 19999)


In [21]:
# verificamos la posición del actuador
retCode,pos=sim.simxGetObjectPosition(clientID, effector, -1, sim.simx_opmode_blocking)
print(pos)

[0.4999592900276184, 0.49999570846557617, 0.09971171617507935]


In [22]:
# y la orientación
retCode,eul=sim.simxGetObjectOrientation(clientID, effector, -1, sim.simx_opmode_blocking)
print(eul[0]*180/3.1416)
print(eul[1]*180/3.1416)
print(eul[2]*180/3.1416)

0.00341679789427
0.00358105347118
45.0004800027


<h3> Taller 8 </h3>
Utilizando el procedimiento para el cálculo de la cinemática inversa, construya una secuencia de movimientos para alinear los tres cubos, uno sobre los otros. Siga el siguiente procedimiento:<br><br>
1. Obtenga la posición del primer cubo, recójalo desde esa posición y ubíquelo en una posición cualquiera que usted decida. Recuerde activar y desactivar el actuador final con la función setEffector()<br>
2. Obtenga la posición y orientación del segundo cubo, recójalo desde su ubicación y colóquelo sobre el primer cubo, asignando la orientación necesaria en el ángulo gamma, de tal forma que quede perfectamente alineado con el primer cubo.<br>
3. Repita el procedimiento para el tercer cubo, de tal manera que quede sobre el segundo y perfectamente alineado con los otros dos.<br>
<br>
Nota: Considere que la única orientación que puede ser manejada es el ángulo gamma (rotación respecto al eje z, respecto del sistema de coordenadas del mundo).<br>
Sugerencia: obtenga automáticamente la posición y orientación de cada cubo y realice los cálculos a partir de estas coordenadas. Recuerde que las coordenadas que obtendrá serán las coordenadas del centro del objeto.

In [23]:
# Para el primer cubo
clientID = connect(19999)

# obtenemos la posición
print("leyendo la posición objetivo...")
retCode,pos=sim.simxGetObjectPosition(clientID, caja1, -1, sim.simx_opmode_blocking)
print(pos)
# y orientación
retCode,eul=sim.simxGetObjectOrientation(clientID, caja1, -1, sim.simx_opmode_blocking)
print(eul[0]*180/3.1416)
print(eul[1]*180/3.1416)
print(eul[2]*180/3.1416)

# definimos las coordenadas de destino
x = pos[0]
y = pos[1]
z = pos[2] + 0.026 # distancia del centro al borde + tolerancia
alpha = eul[0]
beta = eul[1]
gamma = eul[2]
t = sp.Matrix([[1, 0, 0, x],
               [0, 1, 0, y], 
               [0, 0, 1, z], 
               [0, 0, 0, 1]])

D = t*matrixFromEuler(alpha, beta, gamma)

# calculamos la cinemática inversa
print("calculando la cinemática inversa...")
try:
    q = sp.nsolve((T-D), (q1, q2, q3, q4), (1, 1, 1, 1), prec=6)
except:
    print('no se encontró la solución')
    q = [0, 0, 0, 0]
print q

# movemos el robot a la posición
print("moviendo a la posición objetivo...")
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint4, q[3], sim.simx_opmode_blocking)
time.sleep(1)
# bajamos el actuador
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_blocking)
time.sleep(1)
# activamos el efector
setEffector(1)
time.sleep(1)
# levantamos el actuador
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
time.sleep(1)
# volvemos a la posición inicial
retCode = sim.simxSetJointTargetPosition(clientID, joint1, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
time.sleep(1)
# bajamos el cubo
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_blocking)
time.sleep(1)
# desactivamos el actuador
time.sleep(1)
setEffector(0)
# y levantamos
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
print("movimiento concluido!")

('conectado a', 19999)
leyendo la posición objetivo...
[0.5677583813667297, -0.0583653561770916, 0.02499992400407791]
8.76847054349e-07
-8.96514744286e-07
67.1515103239
calculando la cinemática inversa...
Matrix([[-0.870443], [1.71224], [0.183000], [0.330220]])
moviendo a la posición objetivo...
movimiento concluido!
